In [90]:
from Bio import SeqIO
from Bio.Seq import Seq
import csv
import pandas as pd
from collections import Counter
import csv
from Bio.Seq import Seq
import sys
sys.path.append('/home/jupyter/gitsss/yo/tao')
# from myprimer import *
from myfunc import *


In [91]:
cd /home/jupyter/Documents/Lineage/miseqruns/

/home/jupyter/Documents/Lineage/miseqruns


In [92]:
ls

221208_M07962_0015_000000000-DH5TJ/
221208_M07962_0015_000000000-DH5TJ-20221209T185521Z-001.zip
221208_M07962_0015_000000000-DH5TJ.csv
221208_M07962_0015_000000000-DH5TJ.fastqs/
221208_M07962_0015_000000000-DH5TJ.qc.xlsx
221212/


In [52]:
runname='221208_M07962_0015_000000000-DH5TJ'

# read fastq.gz


In [57]:

# import required module
import os
 
# assign directory
dirName = runname+'.fastqs'
 
listOfFiles = list()
for (dirpath, dirnames, filenames) in os.walk(dirName):
    listOfFiles += [os.path.join(dirpath, file) for file in filenames]

In [58]:
fastqgz=[f for f in listOfFiles if f.endswith('fastq.gz')]

In [84]:
import gzip
from Bio import SeqIO

dig=dict()
for fgz in fastqgz:
    tmp=dict()
    sample="_".join(fgz.split('/')[-1].split('_')[:2])
    plate=fgz.split('/')[-1].split('_')[0]
    row=fgz.split('/')[-1].split('_')[1][0]
    col=int(fgz.split('/')[-1].split('_')[1][1:])
    tmp['plate']=plate
    tmp['row']=row
    tmp['col']=col

    solid_reads=0
    total_reads=0
    with gzip.open(fgz, "rt") as handle:
        for record in SeqIO.parse(handle, "fastq"):
            total_reads+=1
            seq=record.seq
            #get rid of NNs
            #get_rid of TTT, GGG, AAA, CCC
            if len(set(list(seq)))>3 and  "N"*20 not in seq and len(seq)>100:
                solid_reads+=1
    tmp['solid_reads']=solid_reads
    tmp['total_reads']=total_reads
    tmp['success_rate']=solid_reads/total_reads
    try:
        tmp['1000']=round(1000/solid_reads,1)
    except:
        tmp['1000']=0
    dig[sample]=tmp
    

In [85]:
dftt=pd.DataFrame.from_dict(dig,orient='index')
dftt['10000']=dftt['1000']*10

In [89]:
outname=runname+'.qc.xlsx'
dftt.sort_values(by = ["plate",'col', 'row'], ascending = [True,True, True], na_position = 'first', inplace=True)
dftt
dftt.to_excel(outname)